# Título   Crear clase sample

## Autor: Jose Chelquer
## Fecha de última modificación: 11/11/2024
## Descripción:

Crear un sample del dataset con proporciones parametrizadas y meses seleccionados

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [36]:
vm=True              # Se está trabajando en la vm o local?
usar_gdrive=not(vm)  # en vm no se usa google drive

feature_a_usar = 'clase_ternaria'     #feature a la que hacen referencia los valores de % a seleccionar
sample_pct={'BAJA+1':1, 'BAJA+2':1, 'CONTINUA':.05}   # % de cada clase a obtener en el sample
meses_a_seleccionar = [202101, 202102, 202103, 202104, 202105, 202106]       # Versión para seleccionar meses específicos
#meses_a_seleccionar=[]                                # Versión para seleccionar todos los meses
mes_test=202108         #el mes test va completo. SIEMPRE se lo selecciona

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [37]:
dataset_path = '~/buckets/b1/datasets' 
#dataset_file='competencia_02.csv.gz'
dataset_file='competencia_02_aumentada.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [38]:
#output_file='competencia_02_sampleado.csv.gz'
#output_file='sample_c02_base_m1a8.csv.gz'
output_file='sample_c02_aum_m1a8.csv.gz'


## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [39]:
import os
import pandas as pd

In [40]:
if usar_gdrive:
    print ('Cargando google drive')
    from google.colab import drive
    drive.mount('/content/drive')

In [41]:
input_gz=dataset_file.endswith('.gz')
output_gz=output_file.endswith('.gz')

In [42]:
# leer el dataset
file_path = os.path.join(dataset_path, dataset_file)
print (f'leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

leyendo ~/buckets/b1/datasets/competencia_02_aumentada.csv.gz


In [43]:
def contabilizar(df):
    print (f'Total de filas y features: \n{df.shape}')
    print (f'Filas por mes:\n {df['foto_mes'].value_counts()}')
    print (f'Casos por mes y clase:\n{pd.crosstab(df['foto_mes'], df['clase_ternaria'])}')
contabilizar(df)

Total de filas y features: 
(4735593, 485)
Filas por mes:
 foto_mes
202108    165442
202107    165152
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
202012    161526
202011    160742
202010    159731
202009    158371
202008    157058
202007    155764
202006    153757
202005    151261
202004    149872
202003    149356
202002    147109
202001    143966
201912    140661
201911    138667
201910    136682
201909    134314
201908    132664
201907    130724
201906    129186
201905    127659
201904    126996
201903    126436
201902    125799
201901    124752
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
201901             688     718    123346
201902             720     688    124391
201903             688     760    124988
201904             759     579    125658
201905             580     660    126419
201906             662     608    127916
201907     

In [44]:
df_out=df

In [45]:
# Seleccionar meses
if meses_a_seleccionar:
    print (f'seleccionando meses {meses_a_seleccionar}')
    df_out=df_out[df_out['foto_mes'].isin(meses_a_seleccionar)]
    print (f'quedaron {df_out.shape[0]} filas')
contabilizar(df_out)

seleccionando meses [202101, 202102, 202103, 202104, 202105, 202106]
quedaron 981946 filas
Total de filas y features: 
(981946, 485)
Filas por mes:
 foto_mes
202106    164876
202105    164623
202104    164090
202103    163685
202102    162646
202101    162026
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202101             635     785    160606
202102             785    1017    160844
202103            1020     981    161684
202104             982    1189    161919
202105            1189     911    162523
202106             908    1074    162894


In [46]:
# muestrar
# Función para filtrar y muestrear por cada valor del feature
def filtrar_muestrear(df, feature, feat_grupos, sample_pct):
    grupos=df[feat_grupos].unique()
    print (grupos)
    df_sampled = pd.concat([
        df[(df[feature] == clase) & (df[feat_grupos]==grupo)].sample(frac=sample_pct[clase], random_state=42)
        for clase in sample_pct
        for grupo in grupos
    ])
    return df_sampled




# Aplicamos la función al DataFrame
df_out = filtrar_muestrear(df_out, feature_a_usar, 'foto_mes', sample_pct)

print (f'quedaron {df_out.shape[0]} filas')


[202104 202105 202106 202101 202102 202103]
quedaron 59999 filas


In [47]:
contabilizar(df_out)

Total de filas y features: 
(59999, 485)
Filas por mes:
 foto_mes
202104    10267
202105    10226
202106    10127
202103    10085
202102     9844
202101     9450
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202101             635     785      8030
202102             785    1017      8042
202103            1020     981      8084
202104             982    1189      8096
202105            1189     911      8126
202106             908    1074      8145


In [48]:
# Elimino el mes_test si por alguna razón quedó ahí
# Por las dudas, se eliminan registros de ese mes:
df_out = df_out[df_out['foto_mes'] != mes_test]



In [49]:
# Agrego completo el mes_test
# Filtra las filas de df que cumplen la condición
df_mes_test = df[df["foto_mes"] == mes_test]

# Agrega esas filas a df_out
df_out = pd.concat([df_out, df_mes_test], ignore_index=True)

In [50]:
contabilizar(df_out)

Total de filas y features: 
(225441, 485)
Filas por mes:
 foto_mes
202108    165442
202104     10267
202105     10226
202106     10127
202103     10085
202102      9844
202101      9450
Name: count, dtype: int64
Casos por mes y clase:
clase_ternaria  BAJA+1  BAJA+2  CONTINUA
foto_mes                                
202101             635     785      8030
202102             785    1017      8042
202103            1020     981      8084
202104             982    1189      8096
202105            1189     911      8126
202106             908    1074      8145
202108          165442       0         0


In [51]:
output_file_path=os.path.join(dataset_path, output_file)
if output_gz:
    df_out.to_csv(output_file_path, index=False, compression='gzip')
else:
    df_out.to_csv(output_file_path, index=False)
    

In [55]:
# Cantidad de variables
num_variables=len(df.columns)
# Filtra las columnas que empiezan con 'tree'
tree_columns = [col for col in df.columns if col.startswith('tree')]
# Cuenta el número de columnas
num_tree_columns = len(tree_columns)
print (f"Cantidad total de variables {num_variables}")
print(f"Cantidad de columnas que empiezan con 'tree': {num_tree_columns}")
print (f"Cantidad de variables normales: {num_variables-num_tree_columns}")
print (f"Archivo grabado: {output_file}")


Cantidad total de variables 485
Cantidad de columnas que empiezan con 'tree': 0
Cantidad de variables normales: 485
Archivo grabado: sample_c02_aum_m1a8.csv.gz
